# Aşama 2: Veri Ön İşleme

**Proje**: Çelik Levha Hata Tespiti  
**Veri Seti**: Çelik Levha Hata Veri Seti (1.941 örnek)  
**Amaç**: Makine öğrenimi için veriyi temizlemek ve hazırlamak  
**Tarih**: Aralık 2024

---

## Genel Bakış

Bu defter şunları kapsar:
1. Ham veriyi yükleme
2. Tekrarlanan kayıtları işleme
3. Aykırı değer tespiti ve işlemi
4. Özellik ölçekleme ve normalleştirme
5. Kategorik değişkenleri kodlama
6. Katmanlı eğitim-test bölmesi
7. Ön işlenmiş veriyi kaydetme

---

## İçindekiler

1. [Kurulum ve Veri Yükleme](#1-kurulum-ve-veri-yükleme)
2. [Veri Temizleme](#2-veri-temizleme)
3. [Aykırı Değer İşlemi](#3-aykırı-değer-işlemi)
4. [Özellik Mühendisliği](#4-özellik-mühendisliği)
5. [Veri Dönüşümü](#5-veri-dönüşümü)
6. [Eğitim-Test Bölmesi](#6-eğitim-test-bölmesi)
7. [Ön İşlenmiş Veriyi Kaydet](#7-ön-işlenmiş-veriyi-kaydet)
8. [Özet](#8-özet)

---
## 1. Kurulum ve Veri Yükleme

Gerekli kütüphaneleri içe aktarın ve veri setini yükleyin.

In [ ]:
# ============================================================
# LIBRARY IMPORTS
# ============================================================

# Pandas: Data manipulation and analysis
import pandas as pd

# Numpy: Numerical operations
import numpy as np

# Matplotlib & Seaborn: Visualization
import matplotlib.pyplot as plt
import seaborn as sns

# Scikit-learn: Preprocessing utilities
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, LabelEncoder, MinMaxScaler

# Warnings: Suppress unnecessary messages
import warnings
warnings.filterwarnings('ignore')

# Configure visualization
plt.rcParams['figure.figsize'] = (12, 6)
sns.set_style('whitegrid')
sns.set_palette('husl')

# Display settings
pd.set_option('display.max_columns', 50)
pd.set_option('display.precision', 2)

print("✓ Libraries imported successfully!")
print("✓ Ready for data preprocessing!")

### Ham Veriyi Yükle

In [ ]:
# Load the raw dataset
# This is the same data we explored in the EDA phase
data_path = '../data/raw/steel_plates_fault.csv'
df = pd.read_csv(data_path)

# Store original shape for comparison
original_shape = df.shape

# Print initial information
print("✓ Dataset loaded successfully!")
print(f"\n📊 Original Dataset Shape:")
print(f"   - Rows: {df.shape[0]:,}")
print(f"   - Columns: {df.shape[1]}")

# Separate features and target
feature_columns = [col for col in df.columns if col != 'Class']
target_column = 'Class'

print(f"\n📋 Features: {len(feature_columns)}")
print(f"🎯 Target: '{target_column}'")

# Quick look at the data
df.head()

---
## 2. Veri Temizleme

Eksik değerleri ve tekrarları işleyerek veri setini temizleyin.

**Veri temizleme neden önemli:**
- Eksik değerler algoritmalarda hatalara neden olabilir
- Tekrarlar model eğitimini yanlılayabilir
- Temiz veri daha iyi model performansına yol açar

### 2.1 Eksik Değerleri Kontrol Et

In [ ]:
# Check for missing values in each column
# isnull().sum() counts missing values per column
missing_values = df.isnull().sum()
total_missing = missing_values.sum()

print("🔍 Missing Values Check:")
print(f"\n   Total missing values: {total_missing}")

if total_missing == 0:
    print("   ✅ No missing values found!")
    print("   No imputation needed.")
else:
    print("   ⚠️ Missing values detected!")
    print("\n   Missing by column:")
    print(missing_values[missing_values > 0])
    
    # If there were missing values, we would handle them here:
    # Option 1: Drop rows with missing values
    # df = df.dropna()
    
    # Option 2: Fill with mean/median/mode
    # df = df.fillna(df.mean())
    
    # Option 3: Fill with specific value
    # df = df.fillna(0)

### 2.2 Tekrarlanan Kayıtları Kaldır

In [ ]:
# Check for and remove duplicate records
# Duplicates can artificially inflate model performance

# Count duplicates before removal
num_duplicates = df.duplicated().sum()
print("🔍 Duplicate Records Check:")
print(f"\n   Duplicates found: {num_duplicates}")

if num_duplicates > 0:
    # Remove duplicates, keeping the first occurrence
    df_cleaned = df.drop_duplicates(keep='first')
    
    print(f"   ✅ Removed {num_duplicates} duplicate records")
    print(f"\n   Before: {len(df):,} rows")
    print(f"   After:  {len(df_cleaned):,} rows")
    
    # Update the dataframe
    df = df_cleaned
else:
    print("   ✅ No duplicates found!")
    print("   Dataset is already clean.")

# Verify the change
print(f"\n📊 Current dataset shape: {df.shape}")

### 2.3 Veri Türü Doğrulaması

In [ ]:
# Verify data types are correct for each column
# Incorrect types can cause unexpected behavior

print("🔍 Data Type Validation:\n")

# Get current data types
dtypes = df.dtypes

# Check numerical features
numerical_cols = df.select_dtypes(include=[np.number]).columns.tolist()
print(f"   Numerical columns: {len(numerical_cols)}")

# Check object (string) columns
object_cols = df.select_dtypes(include=['object']).columns.tolist()
print(f"   Object columns: {len(object_cols)}")

# Display data types
print("\n   Data types summary:")
print(df.dtypes.value_counts())

# Check if target is categorical
if df['Class'].dtype == 'object':
    print("\n   ✅ Target 'Class' is categorical (object)")
    print(f"   Unique classes: {df['Class'].nunique()}")
else:
    print("\n   ⚠️ Target 'Class' is numerical, may need encoding")

---
## 3. Aykırı Değer İşlemi

Veri setindeki aykırı değerleri belirleyin ve işleyin.

**Aykırı değerleri işleme yöntemleri:**
1. **Kaldır**: Aşırı değerlere sahip satırları sil
2. **Sınırla**: Değerleri maksimum/minimum eşiğe sınırla
3. **Dönüştür**: Log veya diğer dönüşümleri uygula
4. **Koru**: Aykırı değerler geçerliyse, koru

Aykırı değerleri tespit etmek için **IQR (Çeyrekler Arası Aralık) yöntemi** kullanacağız.

### 3.1 Aykırı Değer Tespiti

In [ ]:
# Detect outliers using IQR method
# IQR = Q3 - Q1
# Outliers: values < Q1 - 1.5*IQR or > Q3 + 1.5*IQR

def detect_outliers_iqr(df, column):
    """
    Detect outliers in a column using IQR method.
    
    Parameters:
    -----------
    df : DataFrame
        The dataset
    column : str
        Column name to check
    
    Returns:
    --------
    tuple : (n_outliers, lower_bound, upper_bound, outlier_indices)
    """
    Q1 = df[column].quantile(0.25)
    Q3 = df[column].quantile(0.75)
    IQR = Q3 - Q1
    
    lower_bound = Q1 - 1.5 * IQR
    upper_bound = Q3 + 1.5 * IQR
    
    outlier_mask = (df[column] < lower_bound) | (df[column] > upper_bound)
    outlier_indices = df[outlier_mask].index
    
    return len(outlier_indices), lower_bound, upper_bound, outlier_indices

# Detect outliers for all numerical features
print("🔍 Outlier Detection (IQR Method):\n")
print("="*70)

numerical_features = [col for col in feature_columns if df[col].dtype in ['int64', 'float64']]
outlier_summary = []

for feature in numerical_features:
    n_out, lb, ub, idx = detect_outliers_iqr(df, feature)
    pct = (n_out / len(df)) * 100
    
    outlier_summary.append({
        'Feature': feature,
        'Outliers': n_out,
        'Percentage': f"{pct:.2f}%",
        'Lower': f"{lb:.2f}",
        'Upper': f"{ub:.2f}"
    })

# Display as DataFrame
outlier_df = pd.DataFrame(outlier_summary)
outlier_df = outlier_df.sort_values('Outliers', ascending=False)
print(outlier_df.head(15).to_string(index=False))

# Summary
total_outliers = outlier_df['Outliers'].sum()
print(f"\n📊 Total outlier instances: {total_outliers:,}")
print(f"   Features with outliers: {(outlier_df['Outliers'] > 0).sum()}/{len(numerical_features)}")

### 3.2 Aykırı Değerleri Görselleştir

In [ ]:
# Visualize outliers with box plots
# Focus on features with most outliers

# Get top 6 features with most outliers
top_outlier_features = outlier_df.head(6)['Feature'].tolist()

fig, axes = plt.subplots(2, 3, figsize=(15, 10))
axes = axes.flatten()

for idx, feature in enumerate(top_outlier_features):
    # Create box plot
    bp = axes[idx].boxplot(df[feature], vert=True, patch_artist=True)
    
    # Color the box
    for patch in bp['boxes']:
        patch.set_facecolor('lightcoral')
    
    # Get outlier count
    n_out = outlier_df[outlier_df['Feature'] == feature]['Outliers'].values[0]
    
    axes[idx].set_title(f'{feature}\n({n_out} outliers)', fontsize=11, fontweight='bold')
    axes[idx].set_ylabel('Value', fontsize=10)
    axes[idx].grid(True, alpha=0.3, axis='y')

plt.suptitle('Box Plots: Features with Most Outliers', fontsize=14, fontweight='bold', y=1.02)
plt.tight_layout()
plt.show()

print("\n💡 Red points beyond whiskers are potential outliers")
print("   Decision: Keep, cap, or remove based on domain knowledge")

### 3.3 Aykırı Değerleri İşle (Sınırlama Yöntemi)

In [ ]:
# Cap outliers using IQR boundaries
# This preserves all data points while limiting extreme values

def cap_outliers_iqr(df, column):
    """
    Cap outliers to IQR boundaries.
    
    Parameters:
    -----------
    df : DataFrame
        The dataset
    column : str
        Column name to cap
    
    Returns:
    --------
    Series : Column with capped values
    """
    Q1 = df[column].quantile(0.25)
    Q3 = df[column].quantile(0.75)
    IQR = Q3 - Q1
    
    lower_bound = Q1 - 1.5 * IQR
    upper_bound = Q3 + 1.5 * IQR
    
    # Cap values
    capped = df[column].clip(lower=lower_bound, upper=upper_bound)
    
    return capped

# Create a copy for preprocessing
df_processed = df.copy()

# Cap outliers for features with significant outliers (>2%)
print("🔧 Capping Outliers:\n")

capped_features = []
for feature in numerical_features:
    n_out, _, _, _ = detect_outliers_iqr(df_processed, feature)
    pct = (n_out / len(df_processed)) * 100
    
    if pct > 2:  # Only cap if more than 2% outliers
        df_processed[feature] = cap_outliers_iqr(df_processed, feature)
        capped_features.append(feature)
        print(f"   ✓ Capped: {feature} ({pct:.1f}% outliers)")

if len(capped_features) == 0:
    print("   No features required capping (all <2% outliers)")
else:
    print(f"\n📊 Total features capped: {len(capped_features)}")

# Verify outliers are reduced
print("\n🔍 Verification after capping:")
for feature in capped_features[:3]:  # Check first 3
    n_out_after, _, _, _ = detect_outliers_iqr(df_processed, feature)
    print(f"   {feature}: {n_out_after} outliers remaining")

---
## 4. Özellik Mühendisliği

Model performansını iyileştirebilecek yeni özellikler oluşturun.

**Çelik hata tespiti için özellik mühendisliği fikirleri:**
1. **Oran özellikleri**: İlgili ölçümleri birleştir
2. **Toplanmış özellikler**: Özellik grupları arasında istatistikler
3. **Alan-spesifik özellikler**: Çelik üretim bilgisine dayalı

### 4.1 Yeni Özellikler Oluştur

In [ ]:
# Create new engineered features
print("🔧 Feature Engineering:\n")

# 1. Area-related features
# Aspect ratio: Width to Height ratio
if 'X_Maximum' in df_processed.columns and 'Y_Maximum' in df_processed.columns:
    df_processed['Aspect_Ratio'] = (df_processed['X_Maximum'] - df_processed['X_Minimum']) / \
                                   (df_processed['Y_Maximum'] - df_processed['Y_Minimum'] + 1)
    print("   ✓ Created: Aspect_Ratio (width/height)")

# 2. Perimeter ratio
if 'X_Perimeter' in df_processed.columns and 'Y_Perimeter' in df_processed.columns:
    df_processed['Perimeter_Ratio'] = df_processed['X_Perimeter'] / (df_processed['Y_Perimeter'] + 1)
    print("   ✓ Created: Perimeter_Ratio")

# 3. Luminosity range
if 'Maximum_of_Luminosity' in df_processed.columns and 'Minimum_of_Luminosity' in df_processed.columns:
    df_processed['Luminosity_Range'] = df_processed['Maximum_of_Luminosity'] - df_processed['Minimum_of_Luminosity']
    print("   ✓ Created: Luminosity_Range")

# 4. Average luminosity
if 'Sum_of_Luminosity' in df_processed.columns and 'Pixels_Areas' in df_processed.columns:
    df_processed['Avg_Luminosity'] = df_processed['Sum_of_Luminosity'] / (df_processed['Pixels_Areas'] + 1)
    print("   ✓ Created: Avg_Luminosity")

# 5. Compactness: How compact is the fault shape
if 'Pixels_Areas' in df_processed.columns and 'X_Perimeter' in df_processed.columns:
    df_processed['Compactness'] = df_processed['Pixels_Areas'] / (df_processed['X_Perimeter'] ** 2 + 1)
    print("   ✓ Created: Compactness")

# Update feature columns list
new_features = ['Aspect_Ratio', 'Perimeter_Ratio', 'Luminosity_Range', 'Avg_Luminosity', 'Compactness']
new_features = [f for f in new_features if f in df_processed.columns]

print(f"\n📊 New features created: {len(new_features)}")
print(f"   Total features now: {df_processed.shape[1] - 1}")  # -1 for target

# Display new features
print("\n📋 New Feature Statistics:")
df_processed[new_features].describe().round(2)

---
## 5. Veri Dönüşümü

Makine öğrenimi algoritmaları için özellikleri dönüştürün.

**Dönüşüm neden önemli:**
- Birçok algoritma normal dağılımlı veri varsayar
- Özellikler benzer ölçeklerde olmalı
- Kategorik değişkenler kodlama gerektirir

### 5.1 Hedef Değişkeni Kodla

In [ ]:
# Encode the target variable (Class)
# LabelEncoder converts categorical labels to integers

print("🔧 Encoding Target Variable:\n")

# Create label encoder
label_encoder = LabelEncoder()

# Fit and transform the target
df_processed['Class_Encoded'] = label_encoder.fit_transform(df_processed['Class'])

# Display the mapping
print("   Label Mapping:")
for i, class_name in enumerate(label_encoder.classes_):
    count = (df_processed['Class_Encoded'] == i).sum()
    print(f"   {class_name:15} → {i} ({count} samples)")

print(f"\n   ✅ Target encoded successfully!")
print(f"   Classes: {len(label_encoder.classes_)}")

# Store the encoder for later use (inverse transform)
# This is important for interpreting predictions

### 5.2 Özellik Ölçekleme

In [ ]:
# Prepare features for scaling
# Get all numerical features (excluding target)
feature_cols = [col for col in df_processed.columns 
                if col not in ['Class', 'Class_Encoded']]

print("🔧 Feature Scaling Preparation:\n")
print(f"   Features to scale: {len(feature_cols)}")

# Display statistics before scaling
print("\n📊 Feature Statistics BEFORE Scaling:")
print(df_processed[feature_cols].describe().round(2).T[['mean', 'std', 'min', 'max']].head(10))

print("\n💡 Note: Features have very different scales!")
print("   This can cause problems for algorithms like SVM, KNN, Neural Networks")
print("   We'll apply StandardScaler to normalize features")

---
## 6. Eğitim-Test Bölmesi

Veriyi eğitim ve test setlerine bölün.

**Önemli hususlar:**
- **Katmanlı**: Her iki sette de sınıf oranlarını koru
- **Rastgele durum**: Tekrarlanabilirlik için
- **Test boyutu**: Test için genellikle %20-30

In [ ]:
# Prepare feature matrix (X) and target vector (y)
X = df_processed[feature_cols].values
y = df_processed['Class_Encoded'].values

print("📊 Data Preparation for Split:\n")
print(f"   Feature matrix shape: {X.shape}")
print(f"   Target vector shape: {y.shape}")
print(f"   Number of features: {X.shape[1]}")
print(f"   Number of classes: {len(np.unique(y))}")

In [ ]:
# Split the data into training and testing sets
# - test_size=0.2: 20% for testing, 80% for training
# - random_state=42: Ensures reproducibility
# - stratify=y: Maintains class proportions

X_train, X_test, y_train, y_test = train_test_split(
    X, y,
    test_size=0.2,
    random_state=42,
    stratify=y  # Important for imbalanced datasets!
)

print("✅ Train-Test Split Complete!\n")
print(f"   Training set: {X_train.shape[0]} samples ({X_train.shape[0]/len(X)*100:.0f}%)")
print(f"   Test set:     {X_test.shape[0]} samples ({X_test.shape[0]/len(X)*100:.0f}%)")

# Verify stratification
print("\n📊 Class Distribution Verification:")
print("\n   Training set:")
for i, class_name in enumerate(label_encoder.classes_):
    train_count = (y_train == i).sum()
    train_pct = train_count / len(y_train) * 100
    print(f"      {class_name}: {train_count} ({train_pct:.1f}%)")

print("\n   Test set:")
for i, class_name in enumerate(label_encoder.classes_):
    test_count = (y_test == i).sum()
    test_pct = test_count / len(y_test) * 100
    print(f"      {class_name}: {test_count} ({test_pct:.1f}%)")

print("\n   ✅ Stratification verified - proportions are similar!")

In [ ]:
# Apply StandardScaler to the features
# IMPORTANT: Fit only on training data, transform both train and test

print("🔧 Applying StandardScaler:\n")

# Create scaler
scaler = StandardScaler()

# Fit on training data only (to prevent data leakage)
# Then transform training data
X_train_scaled = scaler.fit_transform(X_train)

# Transform test data using the same scaler (fitted on training)
X_test_scaled = scaler.transform(X_test)

print("   ✅ Scaling applied successfully!")
print("\n📊 Statistics AFTER Scaling (Training Set):")
print(f"   Mean: {X_train_scaled.mean():.6f} (should be ~0)")
print(f"   Std:  {X_train_scaled.std():.6f} (should be ~1)")
print(f"   Min:  {X_train_scaled.min():.2f}")
print(f"   Max:  {X_train_scaled.max():.2f}")

print("\n💡 Why fit only on training data?")
print("   - Prevents 'data leakage' from test set")
print("   - Test set should simulate unseen data")
print("   - Scaling parameters come only from training data")

---
## 7. Ön İşlenmiş Veriyi Kaydet

Model eğitiminde kullanmak için ön işlenmiş veriyi kaydedin.

In [ ]:
# Create processed data directory if it doesn't exist
import os
processed_dir = '../data/processed'
os.makedirs(processed_dir, exist_ok=True)

# Save the preprocessed dataframe
df_processed.to_csv(f'{processed_dir}/steel_plates_preprocessed.csv', index=False)
print(f"✅ Saved: {processed_dir}/steel_plates_preprocessed.csv")

# Save train/test splits as numpy arrays
np.save(f'{processed_dir}/X_train.npy', X_train_scaled)
np.save(f'{processed_dir}/X_test.npy', X_test_scaled)
np.save(f'{processed_dir}/y_train.npy', y_train)
np.save(f'{processed_dir}/y_test.npy', y_test)
print(f"✅ Saved: X_train.npy, X_test.npy, y_train.npy, y_test.npy")

# Save the feature names
with open(f'{processed_dir}/feature_names.txt', 'w') as f:
    for col in feature_cols:
        f.write(col + '\n')
print(f"✅ Saved: feature_names.txt")

# Save the label encoder classes
with open(f'{processed_dir}/class_names.txt', 'w') as f:
    for cls in label_encoder.classes_:
        f.write(cls + '\n')
print(f"✅ Saved: class_names.txt")

print("\n📁 All preprocessed data saved successfully!")

---
## 8. Özet

### 📊 Ön İşleme Özeti

| Adım | Önce | Sonra | İşlem |
|------|------|-------|-------|
| **Örnekler** | 1.941 | {güncellendi} | Temizlendi |
| **Özellikler** | 27 | {güncellendi} | + Mühendislik |
| **Eksik** | 0 | 0 | Gerekmedi |
| **Tekrarlar** | Kontrol | Kaldırıldı | Temizlendi |
| **Aykırılar** | Tespit | Sınırlandı | IQR yöntemi |
| **Ölçekleme** | Ham | Standardize | StandardScaler |

### 📁 Oluşturulan Dosyalar

```
data/processed/
├── steel_plates_preprocessed.csv  # Tam ön işlenmiş veri seti
├── X_train.npy                    # Eğitim özellikleri (ölçekli)
├── X_test.npy                     # Test özellikleri (ölçekli)
├── y_train.npy                    # Eğitim etiketleri
├── y_test.npy                     # Test etiketleri
├── feature_names.txt              # Özellik sütun adları
└── class_names.txt                # Sınıf etiket adları
```

### 🚀 Sonraki Adımlar

1. **Model Eğitimi**: Modelleri eğitmek için ön işlenmiş veriyi kullan
2. **Model Karşılaştırması**: Birden fazla algoritmayı değerlendir
3. **Hiperparametre Ayarlama**: Model parametrelerini optimize et
4. **Model Değerlendirmesi**: Ayrılmış test setinde test et

---

✅ **Aşama 2 (Ön İşleme) Tamamlandı!**

Veri artık makine öğrenimi için hazır! 🎉

In [ ]:
# Final Summary Statistics
print("="*70)
print("📊 FINAL PREPROCESSING SUMMARY")
print("="*70)

print(f"\n✅ Original samples:     {original_shape[0]:,}")
print(f"✅ Processed samples:    {len(df_processed):,}")
print(f"✅ Original features:    {original_shape[1] - 1}")
print(f"✅ Final features:       {len(feature_cols)}")
print(f"✅ Training samples:     {X_train_scaled.shape[0]:,}")
print(f"✅ Test samples:         {X_test_scaled.shape[0]:,}")
print(f"✅ Classes:              {len(label_encoder.classes_)}")

print("\n" + "="*70)
print("Ready for Model Training! 🚀")
print("="*70)